In [1]:
# Autor: Juan Barrera
import Pkg # Importa el administrador de paquetes Pkg
Pkg.add("JuMP") # Instala el paquete JuMP

In [2]:
Pkg.add("GLPK") # Instala el paquete GLPK

In [3]:
using JuMP # Importa el modulo JuMP.
using GLPK # Importa el modulo GLPK para utilizar el solver.

In [4]:
function execute()
    # CONSTRUCCIÓN DEL MODELO
    modelo = Model(with_optimizer(GLPK.Optimizer)) # Instancia un modelo.

    # PARÁMETROS DE PROBLEMA

    p = [ 53 27  96 94 52; 
            56 55  68 67 55; 
            81 34  67 14 66; 
            86  4  73 21 86; 
            25  8  33 61 49; 
            43 46  63 34 53; 
            92 93   6  2 56] # tiempo de trabajo x máquina

    n,m = size(p)

    J = 1:n # trabajos
    M = 1:m # maquinas
    # VARIABLES DE DECISIÓN
    
     @variable(modelo, x[J, J] >= 0, Bin)  

     @variable(modelo, w[J, M] >= 0, Int) 

     @variable(modelo, v[J, M] >= 0, Int) 


    # FUNCIÓN OBJETIVO

     @objective(modelo, Min, sum(v[i,m] + sum(p[j,m] * x[i,j] for j in 1:n ) for i in 1:n) )
    # RESTRICCIONES

    for i in 1:n
    @constraint(modelo, sum(x[i,j] for j in 1:n) == 1)
    end 

    for j in 1:n
    @constraint(modelo, sum(x[i,j] for i in 1:n) == 1)
    end 

    for k in 1:(m-1)   
    @constraint(modelo, w[1,k] == 0)
    end 

    for k in 2:m  
    @constraint(modelo, v[1,k] == sum( sum( p[i,r] * x[1,i] for i in 1:n) for r in 1:(k-1) )    )
    end 
    
    for i in 1:(n-1)
        for k in 1:(m-1)  
        @constraint(modelo, v[i+1,k] + sum( p[j,k] * x[i+1,j] for j in 1:n) + w[i+1,k] == w[i,k] + sum( p[j,k+1] * x[i,j] for j in 1:n) + v[i+1,k+1])
        end 
    end
    
    optimize!(modelo) # Resuelve el  modelo
    termination_status(modelo) # Entrega valor de estado de término del modelo. 1 = óptimo; 2 = infactible.
    objective_value(modelo)
end


execute (generic function with 1 method)

In [ ]:
@time execute()